In [1]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from neuralhydrology.modelzoo.cudalstm import CudaLSTM
from neuralhydrology.utils.config import Config
from neuralhydrology.datasetzoo import get_dataset
from neuralhydrology.datautils.utils import load_scaler

In [2]:
BASIN_ID = 11129
EPOCH = 20

In [3]:
run_dir = Path("./runs/lstm_1607_123349")
config = Config(run_dir / "config.yml")
config.as_dict()

{'batch_size': 512,
 'clip_gradient_norm': 1,
 'commit_hash': 'b2e3136',
 'data_dir': PosixPath('../data/CAMELS_KZ'),
 'dataset': 'generic',
 'device': 'cuda:0',
 'dynamic_inputs': ['prcp',
  'srad',
  't_max',
  't_min',
  'pp_mean',
  'sat_max',
  'hum_mean',
  'dew_min',
  'discharge_prev'],
 'epochs': 20,
 'experiment_name': 'lstm',
 'head': 'regression',
 'hidden_size': 128,
 'img_log_dir': PosixPath('/home/spectre/Projects/ISSAI/Internship/aqua_rate/ML/runs/lstm_1607_123349/img_log'),
 'initial_forget_bias': 3,
 'learning_rate': {0: 0.01, 10: 0.001, 30: 0.0001, 40: 1e-05},
 'log_interval': 5,
 'log_tensorboard': True,
 'loss': 'NSE',
 'metrics': ['NSE', 'KGE'],
 'model': 'cudalstm',
 'num_workers': 8,
 'number_of_basins': 42,
 'optimizer': 'Adam',
 'output_activation': 'linear',
 'output_dropout': 0.4,
 'package_version': '1.10.0',
 'predict_last_n': 1,
 'run_dir': PosixPath('/home/spectre/Projects/ISSAI/Internship/aqua_rate/ML/runs/lstm_1607_123349'),
 'save_validation_results':

In [4]:
model = CudaLSTM(cfg=config)
model = model.eval()
model

CudaLSTM(
  (embedding_net): InputLayer(
    (statics_embedding): Identity()
    (dynamics_embedding): Identity()
  )
  (lstm): LSTM(31, 128)
  (dropout): Dropout(p=0.4, inplace=False)
  (head): Regression(
    (net): Sequential(
      (0): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)

In [5]:
# model.load_state_dict(torch.load(run_dir / f"model_epoch0{EPOCH}.pt"))
# model

In [6]:
# ds = get_dataset(cfg=config, is_train=False, basin=str(BASIN_ID), period="test", scaler=load_scaler(run_dir))
ds = get_dataset(cfg=config, is_train=False, period="test", scaler=load_scaler(run_dir))
loader = DataLoader(ds, batch_size=1, num_workers=0, collate_fn=ds.collate_fn)

The following basins had not enough valid target values to calculate a standard deviation: 11163, 13035, 19013. NSE loss values for this basin will be NaN.


In [7]:
for key, value in next(iter(loader)).items():
    print(f"{key}: {value.shape}")

x_d: torch.Size([1, 365, 9])
y: torch.Size([1, 365, 1])
date: (1, 365)
x_s: torch.Size([1, 22])
per_basin_target_stds: torch.Size([1, 1, 1])


In [8]:
next(iter(loader))

{'x_d': tensor([[[-0.3361, -0.4118, -1.6943,  ...,  0.8827, -1.9821, -0.0423],
          [-0.3361, -0.4448, -1.5761,  ...,  0.5486, -2.1448, -0.0417],
          [-0.3361, -0.3563, -1.6812,  ...,  0.7308, -2.0634, -0.0417],
          ...,
          [-0.3361, -0.4839, -1.2312,  ..., -1.0004, -1.4211, -0.0623],
          [-0.3361, -0.8706, -1.1984,  ..., -0.2715, -1.0914, -0.0617],
          [ 0.1848, -0.7841, -1.2542,  ...,  0.9434, -1.0572, -0.0611]]]),
 'y': tensor([[[    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [  

In [9]:
input_data = next(iter(loader))
# data preprocess
input_data = model.pre_model_hook(input_data, is_train=False)
# forward pass
pred = model(input_data)
pred

{'lstm_output': tensor([[[ 0.0336,  0.0607, -0.4791,  ...,  0.2230, -0.4831,  0.0303],
          [ 0.0691,  0.1002, -0.5768,  ...,  0.3538, -0.6396,  0.0601],
          [ 0.1037,  0.1396, -0.5807,  ...,  0.4011, -0.6777,  0.0825],
          ...,
          [ 0.9128,  0.2959, -0.5222,  ...,  0.4406, -0.5627,  0.1078],
          [ 0.9102,  0.3021, -0.5121,  ...,  0.4282, -0.5646,  0.1049],
          [ 0.9092,  0.3100, -0.4916,  ...,  0.4138, -0.5812,  0.1092]]],
        grad_fn=<TransposeBackward0>),
 'h_n': tensor([[[ 0.9092,  0.3100, -0.4916, -0.9023,  0.0294, -0.7314, -0.5660,
            0.1322,  0.9069,  0.0398,  0.8225, -0.1441, -0.9200,  0.2833,
           -0.1374,  0.2530,  0.0275,  0.2714,  0.3763,  0.6820, -0.9664,
            0.8426,  0.3826,  0.0231, -0.1320,  0.1377,  0.8935,  0.9867,
            0.4234,  0.9536,  0.8646, -0.0248, -0.0499, -0.1800, -0.9742,
            0.3585, -0.1163,  0.0694,  0.0382, -0.7738, -0.7759, -0.0193,
           -0.0343,  0.9116,  0.9093,  0.7228,

In [18]:
torch.cat([input_data["x_d"], input_data["x_s"].unsqueeze(0).expand(1, 365, 22)], dim=2)

tensor([[[-0.3361, -0.4118, -1.6943,  ...,  0.5518, 44.1814,  1.4554],
         [-0.3361, -0.4448, -1.5761,  ...,  0.5518, 44.1814,  1.4554],
         [-0.3361, -0.3563, -1.6812,  ...,  0.5518, 44.1814,  1.4554],
         ...,
         [-0.3361, -0.4839, -1.2312,  ...,  0.5518, 44.1814,  1.4554],
         [-0.3361, -0.8706, -1.1984,  ...,  0.5518, 44.1814,  1.4554],
         [ 0.1848, -0.7841, -1.2542,  ...,  0.5518, 44.1814,  1.4554]]])

In [19]:
torch.cat([input_data["x_d"], input_data["x_s"].unsqueeze(0).expand(1, 365, 22)], dim=2).shape

torch.Size([1, 365, 31])

In [12]:
for key, value in pred.items():
    print(f"{key}: {value.shape}")

lstm_output: torch.Size([1, 365, 128])
h_n: torch.Size([1, 1, 128])
c_n: torch.Size([1, 1, 128])
y_hat: torch.Size([1, 365, 1])


In [17]:
pred["lstm_output"][0][-1]

tensor([ 0.3302, -0.8622,  0.4456,  0.8004,  0.8064,  0.4695, -0.0156, -0.9456,
         0.0363,  0.7083,  0.3701,  0.8856, -0.1846,  0.9357,  0.0619,  0.7804,
        -0.0303, -0.0097,  0.0797, -0.6157,  0.8869,  0.0305, -0.3562,  0.2382,
        -0.0570,  0.2825, -0.6417,  0.0762, -0.0202, -0.9020,  0.1637,  0.8952,
         0.1324,  0.9303, -0.0331,  0.0695,  0.8389,  0.0384,  0.9450, -0.3909,
         0.0634, -0.0948,  0.9687,  0.8514, -0.5178,  0.2931, -0.2874,  0.4473,
         0.8894,  0.1210,  0.1929,  0.0483,  0.7131, -0.2556,  0.5491, -0.9409,
         0.0402, -0.8828,  0.0688, -0.1610,  0.6998, -0.5243,  0.0094,  0.0856,
         0.3037,  0.7061, -0.4616, -0.7923, -0.7079, -0.0659, -0.2330,  0.7062,
         0.0468,  0.7757, -0.5201,  0.0733, -0.3010,  0.7099, -0.2816, -0.4503,
        -0.1664,  0.6229,  0.1145, -0.1931, -0.9740,  0.0204, -0.0964, -0.0347,
        -0.9471, -0.0631, -0.7580, -0.8708,  0.0407, -0.4568,  0.4037,  0.4515,
        -0.0589,  0.8381,  0.0602,  0.04

In [18]:
pred["h_n"][0][0]

tensor([ 0.3302, -0.8622,  0.4456,  0.8004,  0.8064,  0.4695, -0.0156, -0.9456,
         0.0363,  0.7083,  0.3701,  0.8856, -0.1846,  0.9357,  0.0619,  0.7804,
        -0.0303, -0.0097,  0.0797, -0.6157,  0.8869,  0.0305, -0.3562,  0.2382,
        -0.0570,  0.2825, -0.6417,  0.0762, -0.0202, -0.9020,  0.1637,  0.8952,
         0.1324,  0.9303, -0.0331,  0.0695,  0.8389,  0.0384,  0.9450, -0.3909,
         0.0634, -0.0948,  0.9687,  0.8514, -0.5178,  0.2931, -0.2874,  0.4473,
         0.8894,  0.1210,  0.1929,  0.0483,  0.7131, -0.2556,  0.5491, -0.9409,
         0.0402, -0.8828,  0.0688, -0.1610,  0.6998, -0.5243,  0.0094,  0.0856,
         0.3037,  0.7061, -0.4616, -0.7923, -0.7079, -0.0659, -0.2330,  0.7062,
         0.0468,  0.7757, -0.5201,  0.0733, -0.3010,  0.7099, -0.2816, -0.4503,
        -0.1664,  0.6229,  0.1145, -0.1931, -0.9740,  0.0204, -0.0964, -0.0347,
        -0.9471, -0.0631, -0.7580, -0.8708,  0.0407, -0.4568,  0.4037,  0.4515,
        -0.0589,  0.8381,  0.0602,  0.04

In [15]:
ds._per_basin_target_stds

{'11001': tensor([[0.2768]]),
 '11068': tensor([[0.2847]]),
 '11126': tensor([[1.1639]]),
 '11129': tensor([[1.7891]]),
 '11163': tensor([[nan]]),
 '11164': tensor([[1.7698]]),
 '11275': tensor([[0.2989]]),
 '11293': tensor([[0.1148]]),
 '11397': tensor([[0.0529]]),
 '11433': tensor([[0.9503]]),
 '11469': tensor([[0.5810]]),
 '12002': tensor([[0.0804]]),
 '12008': tensor([[0.0070]]),
 '12032': tensor([[0.1136]]),
 '12072': tensor([[0.0450]]),
 '12075': tensor([[0.0667]]),
 '12564': tensor([[0.1507]]),
 '13002': tensor([[0.0118]]),
 '13005': tensor([[0.2477]]),
 '13035': tensor([[nan]]),
 '13038': tensor([[0.0009]]),
 '13048': tensor([[0.1841]]),
 '13064': tensor([[0.1512]]),
 '13090': tensor([[0.2235]]),
 '13091': tensor([[0.3406]]),
 '13115': tensor([[0.0473]]),
 '13128': tensor([[0.1171]]),
 '13221': tensor([[0.1861]]),
 '19013': tensor([[nan]]),
 '19022': tensor([[0.0144]]),
 '19195': tensor([[0.0735]]),
 '19196': tensor([[0.0385]]),
 '19205': tensor([[0.1231]]),
 '19208': tensor([[

In [18]:
stds = set()
for data in loader:
    stds.add(data["per_basin_target_stds"][0][0][0].item())

In [19]:
stds

{0.2767565846443176,
 0.28465375304222107,
 1.1639032363891602,
 1.7891430854797363,
 1.769752860069275,
 0.29886099696159363,
 0.11484294384717941,
 0.05291290581226349,
 0.9503335952758789,
 0.580983579158783,
 0.08041262626647949,
 0.007003425620496273,
 0.11358395218849182,
 0.04498513415455818,
 0.06672142446041107,
 0.15073318779468536,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.011769600212574005,
 0.24772842228412628,
 0.0009323223493993282,
 0.18408404290676117,
 0.15116849541664124,
 0.22350552678108215,
 0.3406287729740143,
 0.04733413830399513,
 0.11710122227668762,
 0.18605253100395203,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 

In [20]:
len(stds)

3327

In [12]:
len(loader)

46032